SELECTION DU MODELE

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:

# Download the dataset
dataset = pd.read_csv('dataset_cleaned.csv')

# Display the first few rows of the dataframe
dataset.head()

,nom_region,nom_espece,superficie,pluviometrie,rendement_moyen,annee,temperature_moyenne,mois_plantation,log_pluviometrie,categorie_pluviometrie
0,Abidjan,Cacao,311.35,221.88,9.40,2019,25.670,4,5.406634,Moyenne
1,Abidjan,Cacao,141.35,221.88,9.40,2019,25.670,8,5.406634,Moyenne
2,Abidjan,Cacao,301.74,221.88,9.40,2019,25.670,3,5.406634,Moyenne
3,Abidjan,Café,114.35,92.11,6.29,2019,29.375,6,4.533782,Faible
4,Abidjan,Café,253.65,92.11,6.29,2019,29.375,12,4.533782,Faible


Preprocessing

In [17]:
import numpy as np
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler



In [18]:
# 1. Séparation des caractéristiques (features) et de la cible (target)
X = dataset.drop('rendement_moyen', axis=1)
y = dataset['rendement_moyen']

# 2. Identification des colonnes numériques et non numériques
numeric_columns = X.select_dtypes(include=['number']).columns
categorical_columns = X.select_dtypes(exclude=['number']).columns

print("Colonnes numériques :", numeric_columns)
print("Colonnes catégoriques :", categorical_columns)

# 3. Création du préprocesseur
#    - Encodage OneHotEncoder pour les colonnes catégoriques.
#    - Standardisation (StandardScaler) pour les colonnes numériques.
preprocessor = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_include=np.number)),  # Standardisation des colonnes numériques
    (OneHotEncoder(), make_column_selector(dtype_exclude=np.number))   # Encodage des colonnes catégoriques
)

# 4. Création du pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# 5. Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Application du pipeline aux données
X_train = pipeline.fit_transform(X_train)  # Ajustement et transformation sur l'ensemble d'entraînement
X_test = pipeline.transform(X_test)        # Transformation uniquement sur l'ensemble de test

Colonnes numériques : Index(['superficie', 'pluviometrie', 'annee', 'temperature_moyenne',
       'mois_plantation', 'log_pluviometrie'],
      dtype='object')
Colonnes catégoriques : Index(['nom_region', 'nom_espece', 'categorie_pluviometrie'], dtype='object')


RFECV pour Validation Croisée Automatique

In [19]:
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV



In [20]:
# Modèle utilisé pour RFECV
model = RandomForestRegressor(random_state=42)

# RFECV pour sélection récursive avec validation croisée
rfecv = RFECV(estimator=model, step=1, cv=5, scoring='r2')  # 5-fold cross-validation
rfecv.fit(X_train, y_train)

# Résultats
print("Nombre optimal de caractéristiques :", rfecv.n_features_)

Nombre optimal de caractéristiques : 35


In [21]:
mean_cv_scores = rfecv.cv_results_['mean_test_score']
print("Scores moyens (R²) pour chaque nombre de caractéristiques :", mean_cv_scores)


Scores moyens (R²) pour chaque nombre de caractéristiques : [0.76783509 0.77201804 0.77213856 0.61083419 0.55879801 0.61838578
 0.6436908  0.67219025 0.71408638 0.72905949 0.72760366 0.74154994
 0.73986543 0.75987806 0.76308018 0.77606944 0.77916442 0.7819086
 0.78764687 0.79147297 0.79826013 0.80286214 0.80654083 0.80619448
 0.80809024 0.81293046 0.81163412 0.81400403 0.8142673  0.81782658
 0.81991561 0.82198333 0.82315639 0.8260233  0.82700429 0.82607352
 0.82435543]


Affichage des Colonnes selectionnées

In [22]:
# Récupérer les noms des colonnes après transformation
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()

# Filtrer les colonnes sélectionnées
selected_features = [feature_names[i] for i in range(len(feature_names)) if rfecv.support_[i]]

# Afficher les colonnes sélectionnées
print("Colonnes sélectionnées par RFECV :")
print(selected_features)



Colonnes sélectionnées par RFECV :
['standardscaler__superficie', 'standardscaler__pluviometrie', 'standardscaler__temperature_moyenne', 'standardscaler__mois_plantation', 'standardscaler__log_pluviometrie', 'onehotencoder__nom_region_Abidjan', 'onehotencoder__nom_region_Bas-Sassandra', 'onehotencoder__nom_region_Comoé', 'onehotencoder__nom_region_Denguélé', 'onehotencoder__nom_region_Gôh-Djiboua', 'onehotencoder__nom_region_Lacs', 'onehotencoder__nom_region_Lagunes', 'onehotencoder__nom_region_Montagnes', 'onehotencoder__nom_region_Sassandra-Marahoué', 'onehotencoder__nom_region_Savanes', 'onehotencoder__nom_region_Vallée du Bandama', 'onehotencoder__nom_region_Woroba', 'onehotencoder__nom_region_Yamoussoukro', 'onehotencoder__nom_region_Zanzan', 'onehotencoder__nom_espece_Acajou', 'onehotencoder__nom_espece_Anacardier', 'onehotencoder__nom_espece_Cacao', 'onehotencoder__nom_espece_Café', 'onehotencoder__nom_espece_Cocotier', 'onehotencoder__nom_espece_Eucalyptus', 'onehotencoder__nom

Réentraîner un modèle avec les caractéristiques sélectionnées

In [23]:
# Réduire les données aux caractéristiques sélectionnées
X_train_selected = rfecv.transform(X_train)
X_test_selected = rfecv.transform(X_test)

# Réentraîner un modèle avec les caractéristiques sélectionnées
final_model = RandomForestRegressor(random_state=42)
final_model.fit(X_train_selected, y_train)

# Évaluer le modèle sur les données de test
y_pred = final_model.predict(X_test_selected)
from sklearn.metrics import mean_squared_error, r2_score
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.4f}")
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")


Mean Squared Error: 0.2530
R² Score: 0.9612


Cross validation pour voir si le modele est stable

In [24]:
# Validation croisée sur le modèle actuel
cv_scores = cross_val_score(final_model, X_train_selected, y_train, cv=5, scoring='r2')

print(f"Validation croisée - Score R² moyen : {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

Validation croisée - Score R² moyen : 0.8270 ± 0.0378


Installation du modele

In [25]:
import joblib

# Sauvegarder le modèle
joblib.dump(final_model, "final_model.pkl")



['final_model.pkl']

In [31]:
# Sauvegarder les colonnes sélectionnées
joblib.dump(X.columns.to_list(), "features_names.pkl")
joblib.dump(list(selected_features), "selected_features.pkl")


['selected_features.pkl']

In [27]:
# Sauvegarder les prétraitement
joblib.dump(preprocessor, "preprocessor.pkl")

['preprocessor.pkl']

Type  : <class 'list'>
